# Concat after Normalization

### recommended!

- 이렇게 해야지만 feature가 누락되지 않고 데이터가 살아있음

In [2]:
import random
import pandas as pd 
import numpy as np
from numpy import savez_compressed
from numpy import load
from natsort import natsorted
import os

import tensorflow as tf
import keras
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler, StandardScaler ,RobustScaler
from sklearn.model_selection import KFold

from tqdm.notebook import trange, tqdm
from os.path import join

from pickle import dump, load

seed_rand = 777 # 2nd 777 # 1st 41
nameDataset = "IWALQQ_2nd"

In [3]:
def makeColumnsWOMAG():
    SIDEIDX = ['non','oa']
    PARTIDX = ['shank','shoe','thigh']
    TYPEIDX = ['ACC','GYRO','MAG']
    AXISIDX = ['X', 'Y', 'Z']
    LEGCOLUMNSLENGTH = 54
    COl_imu_legs = [f'{SIDEIDX[int(i//(LEGCOLUMNSLENGTH/2))]}\
_{PARTIDX[(i//(len(TYPEIDX)*len(AXISIDX)))%len(PARTIDX)]}\
_{TYPEIDX[(i//(len(AXISIDX)))%len(TYPEIDX)]}\
_{AXISIDX[i%len(AXISIDX)]}' for i in range(0,LEGCOLUMNSLENGTH)]
    TRKCOLUMNSLENGTH = 9
    Col_imu_trunk = [f'trunk_{TYPEIDX[(i//(len(AXISIDX)))%len(TYPEIDX)]}_{AXISIDX[i%len(AXISIDX)]}' for i in range(0,TRKCOLUMNSLENGTH)]
    # Columns for forceplate
    FPCOLUMNSLENGTH = 12
    FPTYPEIDX = ['GRF','ANGLE','MONM','MOBWHT']
    Col_FP = [f'{FPTYPEIDX[(i//(len(AXISIDX)))%len(FPTYPEIDX)]}_{AXISIDX[i%len(AXISIDX)]}' for i in range(0,FPCOLUMNSLENGTH)]
    # 최종 column
    newColumns = COl_imu_legs+Col_imu_trunk+Col_FP
    newColumnswithoutMAG = [col for col in newColumns if not 'MAG' in col] 
    return newColumnswithoutMAG

# subject로 분류한 것을 다시 전체 데이터 index 기준으로 변경하기
# subject로 Kfold로 된것을 subject fold로 바꿔주는 함수
def kfold2subfold(arrName,listData,train,test):
    arrTrain = []
    arrTest = []
    for pID in arrName[train]:
        idxofID = listData.index[listData['patientID']==pID].copy()
        arrTrain.extend(idxofID.to_list())

    for pID in arrName[test]:
        idxofID = listData.index[listData['patientID']==pID].copy()
        arrTest.extend(idxofID.to_list())
    return arrTrain, arrTest

# local 설정

In [4]:
# 필요한 dir 설정
dataDir =     r'Z:\PROJECTS\iwalqq\Data\V3D\Output\IMU Deep Learning\Data\20220325_raw_byDeepak_csv\INC_ByStep\INC_ByZero\Included_checked'
normalizedDir = join(dataDir, r'NORM')
#######################################################
# 설정창
# 이번에는 time-normalized 한 data를 씀 무슨 데이터 길이든 101로 만든 것
TargetDir = normalizedDir
#######################################################
# 파일목록 가져오기
# 파일 목록 전체 들고오고 필요한 확장자만 고르고 들고온 파일 목록에서 .txt만 남기고 나머지 것들 제외시키기
dataExt = r".csv"
listFromFolder = natsorted([_ for _ in os.listdir(TargetDir) if _.endswith(dataExt)])
# 파일 정리한 목록 불러오기
listfileName  = r'list_dataset.xlsx'
listFromxlsx = pd.read_excel(join(dataDir,listfileName))
#인원 추출하기
arrName = listFromxlsx.patientID.unique()
print("항상 listfileName의 수와 dataList는 순서와 개수가 일치해야한다")
print(f"\nNum_listFromxlsx: {len(listFromxlsx)} | Num_listFromFolder: {len(listFromFolder)}")
print(f'Is same size: {len(listFromxlsx)==len(listFromFolder)}')
listFromxlsx.head(), listFromFolder[:5]

항상 listfileName의 수와 dataList는 순서와 개수가 일치해야한다

Num_listFromxlsx: 877 | Num_listFromFolder: 877
Is same size: True


(  patientID  dateVisit speed  numtrial    side  numStep
 0      P002      31220     w         7  nonleg        1
 1      P002      31220     w         7  nonleg        2
 2      P002      31220     w         7   oaleg        1
 3      P002      31220     w         8  nonleg        1
 4      P002      31220     w         8   oaleg        1,
 ['N_F_P002_031220_w_0007_nonleg_imu_knee_angle_moment_1_Step.csv',
  'N_F_P002_031220_w_0007_nonleg_imu_knee_angle_moment_2_Step.csv',
  'N_F_P002_031220_w_0007_oaleg_imu_knee_angle_moment_1_Step.csv',
  'N_F_P002_031220_w_0008_nonleg_imu_knee_angle_moment_1_Step.csv',
  'N_F_P002_031220_w_0008_oaleg_imu_knee_angle_moment_1_Step.csv'])

# SCC 설정

In [17]:
# 필요한 dir 설정
dataDir =     r'.'
normalizedDir = join(dataDir, r'NORM')
#######################################################
# 설정창
# 이번에는 time-normalized 한 data를 씀 무슨 데이터 길이든 101로 만든 것
TargetDir = normalizedDir
#######################################################
# 파일목록 가져오기
# 파일 목록 전체 들고오고 필요한 확장자만 고르고 들고온 파일 목록에서 .txt만 남기고 나머지 것들 제외시키기
dataExt = r".csv"
listFromFolder = natsorted([_ for _ in os.listdir(TargetDir) if _.endswith(dataExt)])
# 파일 정리한 목록 불러오기
listfileName  = r'list_dataset.xlsx'
listFromxlsx = pd.read_excel(join(dataDir,listfileName))
#인원 추출하기
arrName = listFromxlsx.patientID.unique()
print("항상 listfileName의 수와 dataList는 순서와 개수가 일치해야한다")
print(f"\nNum_listFromxlsx: {len(listFromxlsx)} | Num_listFromFolder: {len(listFromFolder)}")
print(f'Is same size: {len(listFromxlsx)==len(listFromFolder)}')
listFromxlsx.head(), listFromFolder[:5]

항상 listfileName의 수와 dataList는 순서와 개수가 일치해야한다

Num_listFromxlsx: 877 | Num_listFromFolder: 877
Is same size: True


(  patientID  dateVisit speed  numtrial    side  numStep
 0      P002      31220     w         7  nonleg        1
 1      P002      31220     w         7  nonleg        2
 2      P002      31220     w         7   oaleg        1
 3      P002      31220     w         8  nonleg        1
 4      P002      31220     w         8   oaleg        1,
 ['N_F_P002_031220_w_0007_nonleg_imu_knee_angle_moment_1_Step.csv',
  'N_F_P002_031220_w_0007_nonleg_imu_knee_angle_moment_2_Step.csv',
  'N_F_P002_031220_w_0007_oaleg_imu_knee_angle_moment_1_Step.csv',
  'N_F_P002_031220_w_0008_nonleg_imu_knee_angle_moment_1_Step.csv',
  'N_F_P002_031220_w_0008_oaleg_imu_knee_angle_moment_1_Step.csv'])

### 데이터 셋 만들기
- 모든 것보다 데이터 split이 가장 먼저 선행되어야 함!
- 데이터 셋을 train|valid|test(80|10|10)으로 나누기 보다 5-fold cross-validation을 하자

In [18]:
# 필요한 함수 및 class 설정
def make_dir(file_path):
    if not os.path.exists(file_path):
        os.makedirs(file_path)
        
# (N , M , P) shape을 가진 입력 데이터를 scaling 할 때 사용!
# 아래 함수는 전체 데이터 셋에 column-wise로 적용됨!
# 짱 편함..
class MinMaxScaler3D(MinMaxScaler):
    # def fit_transform(self, X, y=None):
    #     x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
    #     return np.reshape(super().fit_transform(x, y=y), newshape=X.shape)
    def fit(self, X, y=None):
        x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
        super().fit(x, y=y)

    def transform(self, X):
        x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
        return np.reshape(super().transform(x), newshape=X.shape)
    
    def inverse_transform(self, X):
        x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
        return np.reshape(super().inverse_transform(x), newshape=X.shape)

class StandardScaler3D(StandardScaler):
    # def fit_transform(self, X, y=None):
    #     x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
    #     return np.reshape(super().fit_transform(x, y=y), newshape=X.shape)

    def fit(self, X, y=None):
        x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
        super().fit(x, y=y)

    def transform(self, X):
        x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
        return np.reshape(super().transform(x), newshape=X.shape)
    
    def inverse_transform(self, X):
        x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
        return np.reshape(super().inverse_transform(x), newshape=X.shape)

class RobustScaler3D(RobustScaler):
    # def fit_transform(self, X, y=None):
    #     x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
    #     return np.reshape(super().fit_transform(x, y=y), newshape=X.shape)

    def fit(self, X, y=None):
        x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
        super().fit(x, y=y)

    def transform(self, X):
        x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
        return np.reshape(super().transform(x), newshape=X.shape)
    
    def inverse_transform(self, X):
        x = np.reshape(X, newshape=(X.shape[0]*X.shape[1], X.shape[2]))
        return np.reshape(super().inverse_transform(x), newshape=X.shape)

# 전체 K-fold 용
- 본 cell을 열어서 돌리면 K-fold로 된 K 쌍의 (train set,test set, fitted scaler) 이 저장됨

In [19]:
# KFOLD 선언
kfold = KFold(n_splits=5, random_state=seed_rand, shuffle=True)
# 항상 피험자 명단을 kfold.split(여기)에 넣어야 됨
countfold = -1
print(f"Total subject:{len(arrName)}")
print(f"Total Data size:{len(listFromxlsx)}")
for train,test in kfold.split(arrName):
    # 해당 fold numbering
    countfold = countfold + 1
    # 데이터 셋 만들기
    # 이번 학습에 사용되는 피험자 명단
    print("+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+")
    print(f"Num. of fold: {countfold}\n")
    print(f'{arrName[train]}\nNum for train:{len(arrName[train])}\n')
    print(f'{arrName[test]}\nNum for test:{len(arrName[test])}\n')
    print(f'Num total:{len(arrName[train])+len(arrName[test])}')

    idx4train,idx4test = kfold2subfold(arrName,listFromxlsx,train,test)
    print(f"\n{countfold}_fold\nidx4train:{len(idx4train)} | idx4test:{len(idx4test)} | total:{len(idx4train)+len(idx4test)}/{len(listFromxlsx)}")
    # 전체 목록을 한번에 만들고 필요할 때마다 쓰도록 세팅
    # 전체 데이터를 담고, 이걸 피험자 별로 추출할 수 있도록 만들기
    columnsWOMAG = makeColumnsWOMAG()
    X_columns = [str(i) for i in range(0,42)]
    df_trainData_X = pd.DataFrame(columns=X_columns)

    Y_columns = [str(i) for i in range(0,3)]
    df_trainData_Y_angle = pd.DataFrame(columns=Y_columns) # angle은 3축
    df_trainData_Y_moBWHT = pd.DataFrame(columns=Y_columns) # moment도 3축
    
    # Train 데이터에 해당하는 것만 담기!
    # 아래와 동일하게 valid와 test 셋도 만들 수 있음!
    for idx, datum in enumerate(tqdm([listFromFolder[i] for i in idx4train])):
        df = pd.read_csv(join(TargetDir,datum))
        # 데이터에서 일단 MAG 모두 제외하기
        dfWOMAG = df.loc[:,columnsWOMAG]
        # 측정된 moment 다리가 nonleg이면 그대로 file 두기
        if listFromxlsx.loc[idx,'side'] == "oaleg":
            targetLegArr = dfWOMAG.loc[:,'oa_shank_ACC_X':'oa_thigh_GYRO_Z']
            nonTargetLegArr = dfWOMAG.loc[:,'non_shank_ACC_X':'non_thigh_GYRO_Z'] # mag 빼기
            otherArr = dfWOMAG.loc[:,'trunk_ACC_X':'trunk_GYRO_Z'] 
        else:
            targetLegArr = dfWOMAG.loc[:,'non_shank_ACC_X':'non_thigh_GYRO_Z'] # mag 빼기
            nonTargetLegArr = dfWOMAG.loc[:,'oa_shank_ACC_X':'oa_thigh_GYRO_Z'] # mag 빼기
            otherArr = dfWOMAG.loc[:,'trunk_ACC_X':'trunk_GYRO_Z']
        # 데이터를 항상 동일한 순서로 만들기
        concated = pd.concat([targetLegArr, nonTargetLegArr,otherArr],axis=1)

        # rename하기
        # 쌓아야 하니까..
        concated.columns = X_columns
        # input data 누적
        df_trainData_X = pd.concat([df_trainData_X, concated],axis=0,ignore_index=True)
        ##############################################################
        # output data 만들기
        # kinematic(Angle)
        angle = dfWOMAG.loc[:,'ANGLE_X':'ANGLE_Z']
        angle.columns = Y_columns
        # output data 누적
        df_trainData_Y_angle = pd.concat([df_trainData_Y_angle, angle],axis=0,ignore_index=True)
        # kinetic(moment)
        moBWHT = dfWOMAG.loc[:,'MOBWHT_X':'MOBWHT_Z']
        moBWHT.columns = Y_columns
        # output data 누적
        df_trainData_Y_moBWHT = pd.concat([df_trainData_Y_moBWHT, moBWHT],axis=0,ignore_index=True)
    # Scaler 적용
    # 현재는 MInMaxscaler3D만 적용
    scaler4X = MinMaxScaler() # Z-score nomalization
    scaler4Y_angle = MinMaxScaler() # Z-score nomalization
    scaler4Y_moBWHT  = MinMaxScaler() # Z-score nomalization

    scaler4X.fit(df_trainData_X) # ONLY FOR TRAIN DATA!!!!ONLY FOR TRAIN DATA!!!!ONLY FOR TRAIN DATA!!!!
    scaler4Y_angle.fit(df_trainData_Y_angle) # ONLY FOR TRAIN DATA!!!!ONLY FOR TRAIN DATA!!!!ONLY FOR TRAIN DATA!!!!
    scaler4Y_moBWHT.fit(df_trainData_Y_moBWHT) # ONLY FOR TRAIN DATA!!!!ONLY FOR TRAIN DATA!!!!ONLY FOR TRAIN DATA!!!!

    scaled_X_train = scaler4X.transform(df_trainData_X)
    scaled_Y_angle_train = scaler4Y_angle.transform(df_trainData_Y_angle)
    scaled_Y_moBWHT_train = scaler4Y_moBWHT.transform(df_trainData_Y_moBWHT)

    # scaler 저장 위치
    scalerDir = join(dataDir, r'SAVE_dataSet',nameDataset)
    make_dir(scalerDir)
    # scaler 저장
    dump(scaler4X, open(join(scalerDir,f"{countfold}_fold_scaler4X.pkl"), 'wb'))
    dump(scaler4Y_angle, open(join(scalerDir,f'{countfold}_fold_scaler4Y_angle.pkl'), 'wb'))
    dump(scaler4Y_moBWHT, open(join(scalerDir,f'{countfold}_fold_scaler4Y_moBWHT.pkl'), 'wb'))

    # 이제 제대로 scaling 했으니까 원상복구 하자!
    # 원하는 shape 형태 
    # (N, 4242, 1), (N, 303, 1), (N, 303, 1)   N 은 데이터 수
    X_train = []
    Y_angle_train = []
    Y_moBWHT_train = []
    for i in range(0,len(idx4train)):
        chopped_X_train = scaled_X_train[i*101:101+i*101,:]
        X_train.append(chopped_X_train.flatten('F').reshape(-1,1))
        
        chopped_Y_angle_train= scaled_Y_angle_train[i*101:101+i*101,:]
        Y_angle_train.append(chopped_Y_angle_train.flatten('F').reshape(-1,1))

        chopped_Y_moBWHT_train= scaled_Y_moBWHT_train[i*101:101+i*101,:]
        Y_moBWHT_train.append(chopped_Y_moBWHT_train.flatten('F').reshape(-1,1))

    final_X_train = np.array(X_train)
    final_Y_angle_train = np.array(Y_angle_train)
    final_Y_moBWHT_train = np.array(Y_moBWHT_train)

    # 만들어진 데이터 shape 확인! 
    print(f'TRAIN data  :  {len(idx4train)}')
    print(f'Final shape: {final_X_train.shape}, {final_Y_angle_train.shape}, {final_Y_moBWHT_train.shape}')

    # 데이터 저장 위치
    setDir = join(dataDir, r'SAVE_dataSet',nameDataset)
    make_dir(setDir)
    # 데이터 저장
    savez_compressed(join(setDir,f"{countfold}_fold_final_train.npz"), final_X_train=final_X_train,final_Y_angle_train=final_Y_angle_train,final_Y_moBWHT_train=final_Y_moBWHT_train)
    
    #############################################################################################################################################
    #test set 용
    # 전체 목록을 한번에 만들고 필요할 때마다 쓰도록 세팅
    # 전체 데이터를 담고, 이걸 피험자 별로 추출할 수 있도록 만들기
    columnsWOMAG = makeColumnsWOMAG()
    X_columns = [str(i) for i in range(0,42)]
    df_testData_X = pd.DataFrame(columns=X_columns)

    Y_columns = [str(i) for i in range(0,3)]
    df_testData_Y_angle = pd.DataFrame(columns=Y_columns) # angle은 3축
    df_testData_Y_moBWHT = pd.DataFrame(columns=Y_columns) # moment도 3축
    # test 데이터에 해당하는 것만 담기!
    # 아래와 동일하게 valid와 test 셋도 만들 수 있음!
    for idx, datum in enumerate(tqdm([listFromFolder[i] for i in idx4test])):
        df = pd.read_csv(join(TargetDir,datum))
        # 데이터에서 일단 MAG 모두 제외하기
        dfWOMAG = df.loc[:,columnsWOMAG]
        # 측정된 moment 다리가 nonleg이면 그대로 file 두기
        if listFromxlsx.loc[idx,'side'] == "oaleg":
            targetLegArr = dfWOMAG.loc[:,'oa_shank_ACC_X':'oa_thigh_GYRO_Z']
            nonTargetLegArr = dfWOMAG.loc[:,'non_shank_ACC_X':'non_thigh_GYRO_Z'] # mag 빼기
            otherArr = dfWOMAG.loc[:,'trunk_ACC_X':'trunk_GYRO_Z'] 
        else:
            targetLegArr = dfWOMAG.loc[:,'non_shank_ACC_X':'non_thigh_GYRO_Z'] # mag 빼기
            nonTargetLegArr = dfWOMAG.loc[:,'oa_shank_ACC_X':'oa_thigh_GYRO_Z'] # mag 빼기
            otherArr = dfWOMAG.loc[:,'trunk_ACC_X':'trunk_GYRO_Z']
        # 데이터를 항상 동일한 순서로 만들기
        concated = pd.concat([targetLegArr, nonTargetLegArr,otherArr],axis=1)

        # rename하기
        # 쌓아야 하니까..
        concated.columns = X_columns
        # input data 누적
        df_testData_X = pd.concat([df_testData_X, concated],axis=0,ignore_index=True)
        ##############################################################
        # output data 만들기
        # kinematic(Angle)
        angle = dfWOMAG.loc[:,'ANGLE_X':'ANGLE_Z']
        angle.columns = Y_columns
        # output data 누적
        df_testData_Y_angle = pd.concat([df_testData_Y_angle, angle],axis=0,ignore_index=True)
        # kinetic(moment)
        moBWHT = dfWOMAG.loc[:,'MOBWHT_X':'MOBWHT_Z']
        moBWHT.columns = Y_columns
        # output data 누적
        df_testData_Y_moBWHT = pd.concat([df_testData_Y_moBWHT, moBWHT],axis=0,ignore_index=True)
    
    # Scaler 적용
    # 현재는 MInMaxscaler3D만 적용
    scaled_X_test = scaler4X.transform(df_testData_X)
    scaled_Y_angle_test = scaler4Y_angle.transform(df_testData_Y_angle)
    scaled_Y_moBWHT_test = scaler4Y_moBWHT.transform(df_testData_Y_moBWHT)
    
    # 이제 제대로 scaling 했으니까 원상복구 하자!
    # 원하는 shape 형태 
    # (N, 4242, 1), (N, 303, 1), (N, 303, 1)   N 은 데이터 수
    X_test = []
    Y_angle_test = []
    Y_moBWHT_test = []
    for i in range(0,len(idx4test)):
        chopped_X_test = scaled_X_test[i*101:101+i*101,:]
        X_test.append(chopped_X_test.flatten('F').reshape(-1,1))
        
        chopped_Y_angle_test= scaled_Y_angle_test[i*101:101+i*101,:]
        Y_angle_test.append(chopped_Y_angle_test.flatten('F').reshape(-1,1))

        chopped_Y_moBWHT_test= scaled_Y_moBWHT_test[i*101:101+i*101,:]
        Y_moBWHT_test.append(chopped_Y_moBWHT_test.flatten('F').reshape(-1,1))

    final_X_test = np.array(X_test)
    final_Y_angle_test = np.array(Y_angle_test)
    final_Y_moBWHT_test = np.array(Y_moBWHT_test)

    # 만들어진 데이터 shape 확인! 
    print(f'TEST data  :  {len(idx4test)}')
    print(f'Final shape: {final_X_test.shape}, {final_Y_angle_test.shape}, {final_Y_moBWHT_test.shape}')

    # 데이터 저장 위치
    # train 데이터와 동일
    # 데이터 저장
    savez_compressed(join(setDir,f"{countfold}_fold_final_test.npz"), final_X_test=final_X_test,final_Y_angle_test=final_Y_angle_test,final_Y_moBWHT_test=final_Y_moBWHT_test)
    # 이후에 삭제하든 함수로 만들든 할 것

Total subject:44
Total Data size:877
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+
Num. of fold: 0

['P002' 'P007' 'P017' 'P029' 'P050' 'P061' 'P065' 'P066' 'P069' 'P105'
 'P115' 'P119' 'P121' 'P135' 'P136' 'P147' 'P149' 'P168' 'P169' 'P172'
 'P203' 'P222' 'P225' 'P226' 'P229' 'P243' 'P258' 'P263' 'P266' 'P270'
 'P272' 'P273' 'P277' 'P290' 'P297']
Num for train:35

['P104' 'P106' 'P132' 'P134' 'P142' 'P155' 'P196' 'P205' 'P245']
Num for test:9

Num total:44

0_fold
idx4train:702 | idx4test:175 | total:877/877


  0%|          | 0/702 [00:00<?, ?it/s]

TRAIN data  :  702
Final shape: (702, 4242, 1), (702, 303, 1), (702, 303, 1)


  0%|          | 0/175 [00:00<?, ?it/s]

TEST data  :  175
Final shape: (175, 4242, 1), (175, 303, 1), (175, 303, 1)
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+
Num. of fold: 1

['P007' 'P050' 'P061' 'P065' 'P066' 'P104' 'P105' 'P106' 'P119' 'P121'
 'P132' 'P134' 'P136' 'P142' 'P147' 'P155' 'P168' 'P169' 'P172' 'P196'
 'P203' 'P205' 'P222' 'P225' 'P226' 'P229' 'P243' 'P245' 'P258' 'P263'
 'P270' 'P272' 'P273' 'P277' 'P290']
Num for train:35

['P002' 'P017' 'P029' 'P069' 'P115' 'P135' 'P149' 'P266' 'P297']
Num for test:9

Num total:44

1_fold
idx4train:678 | idx4test:199 | total:877/877


...

TEST data  :  183
Final shape: (183, 4242, 1), (183, 303, 1), (183, 303, 1)
+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+
Num. of fold: 4

['P002' 'P007' 'P017' 'P029' 'P050' 'P061' 'P065' 'P069' 'P104' 'P105'
 'P106' 'P115' 'P119' 'P121' 'P132' 'P134' 'P135' 'P142' 'P147' 'P149'
 'P155' 'P196' 'P203' 'P205' 'P225' 'P226' 'P229' 'P243' 'P245' 'P258'
 'P263' 'P266' 'P273' 'P277' 'P290' 'P297']
Num for train:36

['P066' 'P136' 'P168' 'P169' 'P172' 'P222' 'P270' 'P272']
Num for test:8

Num total:44

4_fold
idx4train:746 | idx4test:131 | total:877/877


  0%|          | 0/746 [00:00<?, ?it/s]

TRAIN data  :  746
Final shape: (746, 4242, 1), (746, 303, 1), (746, 303, 1)


  0%|          | 0/131 [00:00<?, ?it/s]

TEST data  :  131
Final shape: (131, 4242, 1), (131, 303, 1), (131, 303, 1)


# 개별 K-fold 용

In [1]:
# KFOLD 선언
kfold = KFold(n_splits=5, random_state=41, shuffle=True)
# 항상 피험자 명단을 kfold.split(여기)에 넣어야 됨
countfold = -1
for train,test in kfold.split(arrName):
    # 해당 fold numbering
    countfold = countfold + 1
    # 데이터 셋 만들기
    # 이번 학습에 사용되는 피험자 명단
    print(f"Num. of fold: {countfold}")
    print(f'{arrName[train]}\nNum for train:{len(arrName[train])}')
    print(f'{arrName[test]}\nNum for test:{len(arrName[test])}')
    print(f'Num total:{len(arrName[train])+len(arrName[test])}')

    idx4train,idx4test = kfold2subfold(arrName,listFromxlsx,train,test)
    
    break
    # 이후에 삭제하든 함수로 만들든 할 것

NameError: name 'KFold' is not defined

In [35]:
len(idx4train), len(idx4test)

(722, 155)

## Train data

In [ ]:
# 전체 목록을 한번에 만들고 필요할 때마다 쓰도록 세팅
# 전체 데이터를 담고, 이걸 피험자 별로 추출할 수 있도록 만들기
columnsWOMAG = makeColumnsWOMAG()
X_columns = [str(i) for i in range(0,42)]
df_trainData_X = pd.DataFrame(columns=X_columns)

Y_columns = [str(i) for i in range(0,3)]
df_trainData_Y_angle = pd.DataFrame(columns=Y_columns) # angle은 3축
df_trainData_Y_moBWHT = pd.DataFrame(columns=Y_columns) # moment도 3축
# Train 데이터에 해당하는 것만 담기!
# 아래와 동일하게 valid와 test 셋도 만들 수 있음!
for idx, datum in enumerate(tqdm([listFromFolder[i] for i in idx4train])):
    df = pd.read_csv(join(TargetDir,datum))
    # 데이터에서 일단 MAG 모두 제외하기
    dfWOMAG = df.loc[:,columnsWOMAG]
    # 측정된 moment 다리가 nonleg이면 그대로 file 두기
    if listFromxlsx.loc[idx,'side'] == "oaleg":
        targetLegArr = dfWOMAG.loc[:,'oa_shank_ACC_X':'oa_thigh_GYRO_Z']
        nonTargetLegArr = dfWOMAG.loc[:,'non_shank_ACC_X':'non_thigh_GYRO_Z'] # mag 빼기
        otherArr = dfWOMAG.loc[:,'trunk_ACC_X':'trunk_GYRO_Z'] 
    else:
        targetLegArr = dfWOMAG.loc[:,'non_shank_ACC_X':'non_thigh_GYRO_Z'] # mag 빼기
        nonTargetLegArr = dfWOMAG.loc[:,'oa_shank_ACC_X':'oa_thigh_GYRO_Z'] # mag 빼기
        otherArr = dfWOMAG.loc[:,'trunk_ACC_X':'trunk_GYRO_Z']
    # 데이터를 항상 동일한 순서로 만들기
    concated = pd.concat([targetLegArr, nonTargetLegArr,otherArr],axis=1)

    # rename하기
    # 쌓아야 하니까..
    concated.columns = X_columns
    # input data 누적
    df_trainData_X = pd.concat([df_trainData_X, concated],axis=0,ignore_index=True)
    ##############################################################
    # output data 만들기
    # kinematic(Angle)
    angle = dfWOMAG.loc[:,'ANGLE_X':'ANGLE_Z']
    angle.columns = Y_columns
    # output data 누적
    df_trainData_Y_angle = pd.concat([df_trainData_Y_angle, angle],axis=0,ignore_index=True)
    # kinetic(moment)
    moBWHT = dfWOMAG.loc[:,'MOBWHT_X':'MOBWHT_Z']
    moBWHT.columns = Y_columns
    # output data 누적
    df_trainData_Y_moBWHT = pd.concat([df_trainData_Y_moBWHT, moBWHT],axis=0,ignore_index=True)


In [20]:
# Scaler 적용
# 현재는 MInMaxscaler3D만 적용
scaler4X = MinMaxScaler() # Z-score nomalization
scaler4Y_angle = MinMaxScaler() # Z-score nomalization
scaler4Y_moBWHT  = MinMaxScaler() # Z-score nomalization


scaler4X.fit(df_trainData_X) # ONLY FOR TRAIN DATA!!!!ONLY FOR TRAIN DATA!!!!ONLY FOR TRAIN DATA!!!!
scaler4Y_angle.fit(df_trainData_Y_angle) # ONLY FOR TRAIN DATA!!!!ONLY FOR TRAIN DATA!!!!ONLY FOR TRAIN DATA!!!!
scaler4Y_moBWHT.fit(df_trainData_Y_moBWHT) # ONLY FOR TRAIN DATA!!!!ONLY FOR TRAIN DATA!!!!ONLY FOR TRAIN DATA!!!!

scaled_X_train = scaler4X.transform(df_trainData_X)
scaled_Y_angle_train = scaler4Y_angle.transform(df_trainData_Y_angle)
scaled_Y_moBWHT_train = scaler4Y_moBWHT.transform(df_trainData_Y_moBWHT)

# scaler 저장 위치
scalerDir = join(dataDir, r'SAVE_fittedScaler')
make_dir(scalerDir)
# scaler 저장
# https://wooono.tistory.com/360
dump(scaler4X, open(join(scalerDir,f"{countfold}_fold_scaler4X.pkl"), 'wb'))
dump(scaler4Y_angle, open(join(scalerDir,f'{countfold}_fold_scaler4Y_angle.pkl'), 'wb'))
dump(scaler4Y_moBWHT, open(join(scalerDir,f'{countfold}_fold_scaler4Y_moBWHT.pkl'), 'wb'))

In [10]:
# 이제 제대로 scaling 했으니까 원상복구 하자!
# 원하는 shape 형태 
# (N, 4242, 1), (N, 303, 1), (N, 303, 1)   N 은 데이터 수
X_train = []
Y_angle_train = []
Y_moBWHT_train = []
for i in range(0,len(idx4train)):
    chopped_X_train = scaled_X_train[i*101:101+i*101,:]
    X_train.append(chopped_X_train.flatten('F').reshape(-1,1))
    
    chopped_Y_angle_train= scaled_Y_angle_train[i*101:101+i*101,:]
    Y_angle_train.append(chopped_Y_angle_train.flatten('F').reshape(-1,1))

    chopped_Y_moBWHT_train= scaled_Y_moBWHT_train[i*101:101+i*101,:]
    Y_moBWHT_train.append(chopped_Y_moBWHT_train.flatten('F').reshape(-1,1))

final_X_train = np.array(X_train)
final_Y_angle_train = np.array(Y_angle_train)
final_Y_moBWHT_train = np.array(Y_moBWHT_train)

In [30]:
# 만들어진 데이터 shape 확인! 
print(f'TRAIN data  :  {len(idx4train)}')
print(f'Final shape: {final_X_train.shape}, {final_Y_angle_train.shape}, {final_Y_moBWHT_train.shape}')

# 데이터 저장 위치
setDir = join(dataDir, r'SAVE_dataSet')
make_dir(setDir)
# 데이터 저장
# https://machinelearningmastery.com/how-to-save-a-numpy-array-to-file-for-machine-learning/
savez_compressed(join(setDir,f"{countfold}_fold_final_train.npz"), final_X_train=final_X_train,final_Y_angle_train=final_Y_angle_train,final_Y_moBWHT_train=final_Y_moBWHT_train)

TRAIN data  :  722
Final shape: (722, 4242, 1), (722, 303, 1), (722, 303, 1)


In [38]:
# 데이터 불러오기 확인
loaded_train = np.load(join(setDir,f"{countfold}_fold_final_train.npz"))
print(f'loaded Final shape: {loaded_train["final_X_train"].shape}, {loaded_train["final_Y_angle_train"].shape}, {loaded_train["final_Y_moBWHT_train"].shape}')

loaded Final shape: (722, 4242, 1), (722, 303, 1), (722, 303, 1)


In [105]:
# 발표용 데이터
if True:
    pd.DataFrame(final_X_train[0]).to_csv("CBD_IMU_1.csv",index=False)
    pd.DataFrame(final_Y_angle_train[0]).to_csv("CBD_angle_1.csv",index=False)
    pd.DataFrame(final_Y_moBWHT_train[0]).to_csv("CBD_moBWHT_1.csv",index=False)

    reshaped = final_X_train[0].reshape(-1,42, order='F') # 원래 데이터로 복구할 때!! 잘 된것 확인함!
    result = scaler4X.inverse_transform(reshaped)
    pd.DataFrame(result).to_csv("CBD_IMU_rescaled_1.csv",index=False)
    
    reshaped = final_Y_angle_train[0].reshape(-1,3, order='F') # 원래 데이터로 복구할 때!!
    result = scaler4Y_angle.inverse_transform(reshaped)
    pd.DataFrame(result).to_csv("CBD_angle_rescaled_1.csv",index=False)

    reshaped = final_Y_moBWHT_train[0].reshape(-1,3, order='F') # 원래 데이터로 복구할 때!!
    result = scaler4Y_moBWHT.inverse_transform(reshaped)
    pd.DataFrame(result).to_csv("CBD_moBWHT_rescaled_1.csv",index=False)



## Test Data

In [40]:
# 전체 목록을 한번에 만들고 필요할 때마다 쓰도록 세팅
# 전체 데이터를 담고, 이걸 피험자 별로 추출할 수 있도록 만들기
columnsWOMAG = makeColumnsWOMAG()
X_columns = [str(i) for i in range(0,42)]
df_testData_X = pd.DataFrame(columns=X_columns)

Y_columns = [str(i) for i in range(0,3)]
df_testData_Y_angle = pd.DataFrame(columns=Y_columns) # angle은 3축
df_testData_Y_moBWHT = pd.DataFrame(columns=Y_columns) # moment도 3축
# test 데이터에 해당하는 것만 담기!
# 아래와 동일하게 valid와 test 셋도 만들 수 있음!
for idx, datum in enumerate(tqdm([listFromFolder[i] for i in idx4test])):
    df = pd.read_csv(join(TargetDir,datum))
    # 데이터에서 일단 MAG 모두 제외하기
    dfWOMAG = df.loc[:,columnsWOMAG]
    # 측정된 moment 다리가 nonleg이면 그대로 file 두기
    if listFromxlsx.loc[idx,'side'] == "oaleg":
        targetLegArr = dfWOMAG.loc[:,'oa_shank_ACC_X':'oa_thigh_GYRO_Z']
        nonTargetLegArr = dfWOMAG.loc[:,'non_shank_ACC_X':'non_thigh_GYRO_Z'] # mag 빼기
        otherArr = dfWOMAG.loc[:,'trunk_ACC_X':'trunk_GYRO_Z'] 
    else:
        targetLegArr = dfWOMAG.loc[:,'non_shank_ACC_X':'non_thigh_GYRO_Z'] # mag 빼기
        nonTargetLegArr = dfWOMAG.loc[:,'oa_shank_ACC_X':'oa_thigh_GYRO_Z'] # mag 빼기
        otherArr = dfWOMAG.loc[:,'trunk_ACC_X':'trunk_GYRO_Z']
    # 데이터를 항상 동일한 순서로 만들기
    concated = pd.concat([targetLegArr, nonTargetLegArr,otherArr],axis=1)

    # rename하기
    # 쌓아야 하니까..
    concated.columns = X_columns
    # input data 누적
    df_testData_X = pd.concat([df_testData_X, concated],axis=0,ignore_index=True)
    ##############################################################
    # output data 만들기
    # kinematic(Angle)
    angle = dfWOMAG.loc[:,'ANGLE_X':'ANGLE_Z']
    angle.columns = Y_columns
    # output data 누적
    df_testData_Y_angle = pd.concat([df_testData_Y_angle, angle],axis=0,ignore_index=True)
    # kinetic(moment)
    moBWHT = dfWOMAG.loc[:,'MOBWHT_X':'MOBWHT_Z']
    moBWHT.columns = Y_columns
    # output data 누적
    df_testData_Y_moBWHT = pd.concat([df_testData_Y_moBWHT, moBWHT],axis=0,ignore_index=True)

  0%|          | 0/155 [00:00<?, ?it/s]

In [41]:
# Scaler 적용
# 현재는 MInMaxscaler3D만 적용
scaled_X_test = scaler4X.transform(df_testData_X)
scaled_Y_angle_test = scaler4Y_angle.transform(df_testData_Y_angle)
scaled_Y_moBWHT_test = scaler4Y_moBWHT.transform(df_testData_Y_moBWHT)

In [42]:
# 이제 제대로 scaling 했으니까 원상복구 하자!
# 원하는 shape 형태 
# (N, 4242, 1), (N, 303, 1), (N, 303, 1)   N 은 데이터 수
X_test = []
Y_angle_test = []
Y_moBWHT_test = []
for i in range(0,len(idx4test)):
    chopped_X_test = scaled_X_test[i*101:101+i*101,:]
    X_test.append(chopped_X_test.flatten('F').reshape(-1,1))
    
    chopped_Y_angle_test= scaled_Y_angle_test[i*101:101+i*101,:]
    Y_angle_test.append(chopped_Y_angle_test.flatten('F').reshape(-1,1))

    chopped_Y_moBWHT_test= scaled_Y_moBWHT_test[i*101:101+i*101,:]
    Y_moBWHT_test.append(chopped_Y_moBWHT_test.flatten('F').reshape(-1,1))

final_X_test = np.array(X_test)
final_Y_angle_test = np.array(Y_angle_test)
final_Y_moBWHT_test = np.array(Y_moBWHT_test)

In [44]:
# 만들어진 데이터 shape 확인! 
print(f'TEST data  :  {len(idx4test)}')
print(f'Final shape: {final_X_test.shape}, {final_Y_angle_test.shape}, {final_Y_moBWHT_test.shape}')

# 데이터 저장 위치
setDir = join(dataDir, r'SAVE_dataSet')
make_dir(setDir)
# 데이터 저장
savez_compressed(join(setDir,f"{countfold}_fold_final_test.npz"), final_X_test=final_X_test,final_Y_angle_test=final_Y_angle_test,final_Y_moBWHT_test=final_Y_moBWHT_test)

TEST data  :  155
Final shape: (155, 4242, 1), (155, 303, 1), (155, 303, 1)


In [45]:
# 데이터 불러오기 확인
loaded_train = np.load(join(setDir,f"{countfold}_fold_final_test.npz"))
print(f'loaded Final shape: {loaded_train["final_X_test"].shape}, {loaded_train["final_Y_angle_test"].shape}, {loaded_train["final_Y_moBWHT_test"].shape}')

loaded Final shape: (155, 4242, 1), (155, 303, 1), (155, 303, 1)


# 끝

# 새로 배운 것

In [63]:
# reshape 하는 법
flat_concat.shape, concated.to_numpy().reshape(101,42,1).shape

((4242, 1), (101, 42, 1))

In [70]:
a = np.array( [ [1,2,3],[4,5,6],[7,8,9],[10,11,12]])
a[0,:],a.shape


(array([1, 2, 3]), (4, 3))

In [94]:
# reshape은 (행, 렬, 갯수) 순으로 해야 맞음
# concated 는 101행, 42열로 구성됨
concated.to_numpy().reshape(101,42,1)[:,0,0] # 올바른 방향으로 읽어짐

array([-2.08866167, -1.6116709 , -1.12578921, -0.69913567, -0.31552896,
        0.01626456,  0.2934858 ,  0.49830202,  0.62481276,  0.67396727,
        0.66056931,  0.60059968,  0.50080002,  0.36252644,  0.18663478,
       -0.0244365 , -0.26707541, -0.52839363, -0.78795609, -1.01967686,
       -1.20064522, -1.31313809, -1.34727698, -1.30148371, -1.18328261,
       -1.01031294, -0.80652645, -0.59887084, -0.41240249, -0.26469963,
       -0.1639805 , -0.11075129, -0.09801369, -0.11699963, -0.1552758 ,
       -0.19970287, -0.2355143 , -0.24934117, -0.23495442, -0.19335586,
       -0.13348058, -0.066768  , -0.00379263,  0.04791029,  0.08493437,
        0.10705636,  0.1168679 ,  0.11778726,  0.1139358 ,  0.10917843,
        0.10651575,  0.10806219,  0.11395871,  0.12298314,  0.13297936,
        0.14167265,  0.14704517,  0.14762067,  0.14238325,  0.13102786,
        0.11354904,  0.09108217,  0.06584669,  0.04045179,  0.01679698,
       -0.00449873, -0.02328051, -0.03918528, -0.0509166 , -0.05

In [230]:
# train valid test로 두번 쪼개야 할때
arrName = np.array(['P002', 'P007', 'P017', 'P029', 'P050', 'P061', 'P065', 'P066',
       'P069', 'P104', 'P105', 'P106', 'P115', 'P119', 'P121', 'P132',
       'P134', 'P135', 'P136', 'P142', 'P147', 'P149', 'P155', 'P168',
       'P169', 'P172', 'P196', 'P203', 'P205', 'P222', 'P225', 'P226',
       'P229', 'P243', 'P245', 'P258', 'P263', 'P266', 'P270', 'P272',
       'P273', 'P277', 'P290', 'P297'])
# kfold를 위한 사전작업
# subejct 별로 fold가 가능하게 하기
# 일단 KFOLD 선언
kfold = KFold(n_splits=5, random_state=4, shuffle=True)

# 항상 피험자 명단을 kfold.split에 넣어야 됨
for train_whole,test in kfold.split(arrName):
    # 데이터 셋 만들기
    # 이번 학습에 사용되는 피험자 명딘
    print(arrName[train_whole])
    print(arrName[test])
    print(f'{len(arrName[train_whole])},{len(arrName[test])}')
    print("+++++++++++")
    break

arrTWhole = arrName[train_whole]
for train,valid in kfold.split(arrTWhole):
    print(arrTWhole[train])
    print(arrTWhole[valid])
    print(f'{len(arrTWhole[train])},{len(arrName[valid])}')
    break

['P002' 'P007' 'P017' 'P029' 'P061' 'P065' 'P069' 'P104' 'P105' 'P106'
 'P115' 'P119' 'P121' 'P135' 'P147' 'P149' 'P155' 'P168' 'P169' 'P172'
 'P196' 'P203' 'P222' 'P225' 'P229' 'P243' 'P245' 'P258' 'P263' 'P266'
 'P270' 'P272' 'P273' 'P277' 'P290']
['P050' 'P066' 'P132' 'P134' 'P136' 'P142' 'P205' 'P226' 'P297']
35,9
+++++++++++
['P002' 'P007' 'P017' 'P029' 'P061' 'P065' 'P069' 'P104' 'P105' 'P106'
 'P115' 'P119' 'P121' 'P135' 'P147' 'P169' 'P172' 'P203' 'P225' 'P229'
 'P243' 'P245' 'P258' 'P266' 'P270' 'P272' 'P273' 'P290']
['P149' 'P155' 'P168' 'P196' 'P222' 'P263' 'P277']
28,7


In [96]:
# https://stackoverflow.com/questions/50125844/how-to-standard-scale-a-3d-matrix
# gob bless this programmer..
# scaling하는 순서
X_tmp = X_train.copy()
# 선언
scaler = StandardScaler3D()
# fitting
scaler.fit(X_tmp)
# scaler 적용@
scaled_X_tmp = scaler.transform(X_tmp)
# scaler 역변환
rescaeld_X_tmp = scaler.inverse_transform(scaled_X_tmp)